In [1]:
import os
from glob import glob

In [2]:
os.listdir()

['.DS_Store',
 'requirements.txt',
 'README.md',
 'dataloader.py',
 '.ipynb_checkpoints',
 '.git',
 'data',
 'notebooks',
 'Runs.ipynb']

In [31]:
DATA_PATH = "data"

def csv_concat(dir):
    all_files = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            all_files.append(os.path.join(root, name))
            
    csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
    
    return csv_files

csv_files = csv_concat(DATA_PATH)

In [8]:
import pandas as pd

In [9]:
# Read CSV files from List
df = pd.concat(map(pd.read_csv, csv_files))


In [56]:
def file_structuring(csv_files:list()):
    code_file_dict = dict()
    num_file = 0


    for i, f in enumerate(csv_files):
        df = pd.read_csv(f)
        p1, p2, p3 = (df[['notes', 'code', 'desc']], 
                      df[['aug_text', 'code', 'desc']], 
                      df[['paraphrase', 'code', 'desc']])

        p1.columns = ['notes', 'codes', 'desc']
        p2.columns = ['notes', 'codes', 'desc']
        p3.columns = ['notes', 'codes', 'desc']

        df = pd.concat([p1, p2, p3], axis=0, ignore_index=True)
        df = df[['notes', 'codes', 'desc']]
        code_file_dict[i + 1] = df

    df = pd.concat(code_file_dict.values())
    
    return df
    
df = file_structuring(csv_files)
df = df.sample(frac=1)
print(df.shape)
df.head()

(16779, 3)


,notes,codes,desc
20,5. Patient with Other Myositis developed a sev...,M60.8,Other myositis
379,"10. Operative Note: A surgical intervention, a...",M40.1,Other secondary kyphosis
1243,4. Procedure: Arthroscopic hip debridement. In...,M02.0,Arthropathy following intestinal bypass
13,5. Operative Note: The patient with spinal ent...,M46.0,Spinal enthesopathy
3751,"2. synovitis, erosions, and mycotic invasion w...",M01.6,Arthritis in mycoses


In [54]:
df[df.duplicated(['notes'], keep=False)]

,notes,codes,desc
165,6. Operative Note: A joint resurfacing procedu...,M46.2,Osteomyelitis of vertebra
189,The surgical intervention was performed on a p...,M62.3,Immobility syndrome (paraplegic)
207,"""Contracture release surgery was performed on ...",M62.4,Contracture of muscle
48,5. Operative Note: Sacroiliitis Sacroiliac Joi...,M46.1,"Sacroiliitis, not elsewhere classified"
190,There are 9. A surgical intervention was used ...,M46.2,Osteomyelitis of vertebra
...,...,...,...
406,4.,M45,Ankylosing spondylitis
3470,"The procedure consisted of an operation, thoro...",M01.3,Arthritis in other bacterial diseases classifi...
161,1. AS patient diagnosed with mild disease seve...,M45,Ankylosing spondylitis
522,3. A minimally-invagant procedure was carried ...,M40.4,Other lordosis


In [50]:
len(df.codes.unique())

63